<center><h1>Management and Analysis of Physics Dataset (MOD. B) </h1></center>
<center><h2> Project 5 - Streaming processing of cosmic rays using Drift Tubes detectors</h2></center>
<center><h2>Group 2305</h2></center>

<center><style>
    table {font-size: 24px;}
</style></center>

| Last Name        | First Name         |Student ID|
|:----------------:|:------------------:|:--------------:|
| Bertinelli       | Gabriele           |1219907 (tri)   |
| Bhatti           | Roben              |2091187         |
| Bonato           | Diego              |2091250         |
| Cacciola         | Martina            |2097476         |

<left><h2> Part 3 - Live Dashboard</h2></left>

### Import packages and modules 

In [1]:
from bokeh.io import curdoc,show,output_notebook
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, Select, Div
from bokeh.layouts import layout,gridplot,column,row
from bokeh.plotting import figure
from datetime import datetime
from datetime import datetime
from math import radians
import numpy as np
import json
import random

In [2]:
from bokeh.io import curdoc,show,output_notebook
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, Select, Div
from bokeh.layouts import layout,gridplot,column,row
from bokeh.plotting import figure
from datetime import datetime
from datetime import datetime
from math import radians
import numpy as np
import json
import random

import kafka
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka.admin import KafkaAdminClient

### Consumer creation

Subscribing to a topic establishes a connection between the consumer and the partitions of the subscribed topic hosted on the Kafka brokers. This connection allows the consumer to start *polling* for messages from those partitions.

In [3]:
KAFKA_BOOTSTRAP_SERVER = '##'

kafka_admin = KafkaAdminClient(bootstrap_servers = KAFKA_BOOTSTRAP_SERVER)
kafka_admin.list_topics()

['data_clean', 'test_clean_1', 'data_raw', '__consumer_offsets']

In [4]:
# Initialize Kafka consumer by subscribing to the topic
consumer = KafkaConsumer('data_clean',
                         bootstrap_servers = KAFKA_BOOTSTRAP_SERVER)
                        

## Create figure

In [5]:
p1 = figure(width=600, height=600)
p2 = figure(width=350, height=350)
p3 = figure(width=350, height=350)
p4 = figure(width=350, height=350)
p5 = figure(width=350, height=350)
p6 = figure(width=350, height=350)
p7 = figure(width=350, height=350)
p8 = figure(width=350, height=350)
p9 = figure(width=350, height=350)
p10 = figure(width=350, height=350)
p11 = figure(width=350, height=350)
p12 = figure(width=350, height=350)
p13 = figure(width=350, height=350)

source1 = ColumnDataSource(dict(x=[], y=[]))
source2 = ColumnDataSource(dict(x=[], y=[]))
source3 = ColumnDataSource(dict(x=[], y=[]))
source4 = ColumnDataSource(dict(x=[], y=[]))
source5 = ColumnDataSource(dict(x=[], y=[]))
source6 = ColumnDataSource(dict(top=[], left=[], right=[]))
source7 = ColumnDataSource(dict(top=[], left=[], right=[]))
source8 = ColumnDataSource(dict(top=[], left=[], right=[]))
source9 = ColumnDataSource(dict(top=[], left=[], right=[]))
source10 = ColumnDataSource(dict(top=[], left=[], right=[]))
source11 = ColumnDataSource(dict(top=[], left=[], right=[]))
source12 = ColumnDataSource(dict(top=[], left=[], right=[]))
source13 = ColumnDataSource(dict(top=[], left=[], right=[]))


colors = ["#FF0018", "#FF8C00", "#FFEF00", "#008026", "#FF0018", "#FF8C00", "#FFEF00", "#008026", "#FF0018", "#FF8C00", "#FFEF00", "#008026", "#FF0018"]

p1.line(x="x", y="y", line_width=10, color=colors[0], alpha=0.5, source=source1)
p2.line(x="x", y="y", line_width=10, color=colors[1], alpha=0.5, source=source2)
p3.line(x="x", y="y", line_width=10, color=colors[2], alpha=0.5, source=source3)
p4.line(x="x", y="y", line_width=10, color=colors[3], alpha=0.5, source=source4)
p5.line(x="x", y="y", line_width=10, color=colors[4], alpha=0.5, source=source5)
p6.quad(top="top", bottom=0, left="left", right="right", fill_color=colors[0], line_color="black", source=source6)
p7.quad(top="top", bottom=0, left="left", right="right", fill_color=colors[1], line_color="black", source=source7)
p8.quad(top="top", bottom=0, left="left", right="right", fill_color=colors[2], line_color="black", source=source8)
p9.quad(top="top", bottom=0, left="left", right="right", fill_color=colors[3], line_color="black", source=source9)
p10.quad(top="top", bottom=0, left="left", right="right", fill_color=colors[0], line_color=f"{colors[0]}", source=source10)
p11.quad(top="top", bottom=0, left="left", right="right", fill_color=colors[1], line_color=f"{colors[1]}", source=source11)
p12.quad(top="top", bottom=0, left="left", right="right", fill_color=colors[2], line_color=f"{colors[2]}", source=source12)
p13.quad(top="top", bottom=0, left="left", right="right", fill_color=colors[3], line_color=f"{colors[3]}", source=source13)


GlyphRenderer(id='p1752', ...)

 
### Poll messages

In [6]:
def polling():
    for msg in consumer:
        value = json.loads(msg.value.decode('utf-8'))  # Convert the message value to a dictionary
        break
    return value

In [7]:
def create_value():
    
    #instantiate dictionary
    combined_dict = {}
    
    poll = polling()
    
    x = poll["msg_ID"]
    y = poll['hit_count'] 
    combined_dict["p1"] = {"x": [x], "y": [y]}  

    
    for chamber in range(0,4):
        #p2
        x = poll["msg_ID"]
        y = poll['hit_count_chamber'][chamber]
        
        combined_dict[f"p2_{chamber}"] =  {"x": [x], "y": [y]}

        #p3
        chamber_data = poll["tdc_counts_chamber"][f"{chamber}"]
        edges = chamber_data["bin_edges"]
        counts = chamber_data["hist_counts"]
        
        combined_dict[f"p3_{chamber}"] = {"top": counts[:-1], "left": edges[:-1], "right": edges[1:]}


        #p4
        chamber_data = poll["active_tdc_chamber"][f"{chamber}"]
        edges = chamber_data["bin_edges"]
        counts = chamber_data["hist_counts"]

        combined_dict[f"p4_{chamber}"] =  {"top": counts[:-1], "left": edges[:-1], "right": edges[1:]}

    
    return combined_dict

### Create Periodic Function

In [8]:
def update():
    #poll data and filter them
    new_data = create_value()
    
    #stream new data to dashboard
    source1.stream(new_data["p1"], rollover=200)

    source2.stream(new_data["p2_0"], rollover=200)

    source3.stream(new_data["p2_1"], rollover=200)

    source4.stream(new_data["p2_2"], rollover=200)

    source5.stream(new_data["p2_3"], rollover=200)

    source6.data  = new_data["p3_0"]

    source7.data  = new_data["p3_1"]

    source8.data  = new_data["p3_2"]

    source9.data  = new_data["p3_3"]

    source10.data = new_data["p4_0"]

    source11.data = new_data["p4_1"]

    source12.data = new_data["p4_2"]

    source13.data = new_data["p4_3"]


### Format Axis

In [9]:

p1.xaxis.major_label_orientation = radians(80)
p1.xaxis.axis_label = "MSG ID"
p1.yaxis.axis_label = "Number of hits"
p1.title.text = "Total number of hits"


p2.xaxis.major_label_orientation = radians(80)
p2.xaxis.axis_label =  "MSG ID"
p2.yaxis.axis_label = "Number of hits"
p2.title.text = "Chamber 0"

p3.xaxis.major_label_orientation = radians(80)
p3.xaxis.axis_label =  "MSG ID"
p3.yaxis.axis_label = "Number of hits"
p3.title.text = "Chamber 1"

p4.xaxis.major_label_orientation = radians(80)
p4.xaxis.axis_label = "MSG ID"
p4.yaxis.axis_label = "Number of hits"
p4.title.text = "Chamber 2"

p5.xaxis.major_label_orientation = radians(80)
p5.xaxis.axis_label =  "MSG ID"
p4.yaxis.axis_label = "Number of hits"
p5.title.text = "Chamber 3"


p6.xaxis.major_label_orientation = radians(80)
p6.xaxis.axis_label = 'TDC CHANNEL'
p6.yaxis.axis_label = "Number of counts"
p6.title.text = "Chamber 0"

p7.xaxis.major_label_orientation = radians(80)
p7.xaxis.axis_label = 'TDC CHANNEL'
p7.yaxis.axis_label = "Number of counts"
p7.title.text = "Chamber 1"

p8.xaxis.major_label_orientation = radians(80)
p8.xaxis.axis_label = 'TDC CHANNEL'
p8.yaxis.axis_label = "Number of counts"
p8.title.text = "Chamber 2"

p9.xaxis.major_label_orientation = radians(80)
p9.xaxis.axis_label = 'TDC CHANNEL'
p9.yaxis.axis_label = "Number of TDC CHANNEL"
p9.title.text = "Chamber 3"


p10.xaxis.major_label_orientation = radians(80)
p10.xaxis.axis_label = 'ORBIT CNT'
p10.yaxis.axis_label = "Number of channels"
p10.title.text = "Chamber 0"

p11.xaxis.major_label_orientation = radians(80)
p11.xaxis.axis_label = 'ORBIT CNT'
p11.yaxis.axis_label = "Number of channels"
p11.title.text = "Chamber 1"

p12.xaxis.major_label_orientation = radians(80)
p12.xaxis.axis_label = 'ORBIT CNT'
p12.yaxis.axis_label = "Number of channels"
p12.title.text = "Chamber 2"

p13.xaxis.major_label_orientation = radians(80)
p13.xaxis.axis_label = 'ORBIT CNT'
p13.yaxis.axis_label = "Number of channels"
p13.title.text = "Chamber 3"

### Config Layout

In [10]:
square_line_plots = gridplot([[p2, p3], [p4, p5]])

square_line_plots.width = 400
square_line_plots.height = 400

# Margin syntax: [top , right, bottom, left]
p_row1 = [row(p1, square_line_plots, sizing_mode="scale_width",margin=[30,30,50,30])]
p_row2 = [row(p6, p7, p8, p9)]
p_row3 = [row(p10, p11, p12, p13)]

# Create the gridplot
grid = gridplot([p_row1, p_row2, p_row3], toolbar_location=None)

# Add the titles to the respective rows
title_div_row1 = Div(text="<h2>Total number of processed hits</h2>")
title_div_row2 = Div(text="<h2>Total number of hits in each chamber</h2>")
title_div_row3 = Div(text="<h2>Histogram of the counts of active TDC_CHANNEL</h2>")
title_div_row4 = Div(text="<h2>Histogram of the total number of active TDC_CHANNEL in each ORBIT_CNT</h2>")
space = Div(text="<h3> </h3>")

# Create a gridplot with the rows and titles
grid = gridplot([[title_div_row1], p_row1,[space], [title_div_row2], p_row2, [title_div_row3], p_row3], toolbar_location=None, sizing_mode="scale_width")

# Add the overall title to the gridplot
title_div = Div(text="<h1>Cosmic Rays Analysis</h1><p style=\"font-weight: normal;\">Diego Bonato, Martina Cacciola, Gabriele Bertinelli, Roben Bhatti</p>")
grid_with_title = column(title_div, grid, sizing_mode="scale_width")




In [11]:
curdoc().add_root(grid_with_title)
curdoc().add_periodic_callback(update,500)

In [ ]:
!bokeh serve --show consumer_dashboard.ipynb